In [8]:
import os
import time
import pandas as pd
import whisper
import subprocess
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import cv2

In [2]:
df = pd.read_parquet("hf://datasets/lmms-lab/AISG_Challenge/data/test-00000-of-00001.parquet")

/Users/ryan/Documents/GitHub/AISG_Challenge/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df.head()

,qid,video_id,question_type,capability,question,duration,question_prompt,answer,youtube_url
0,0008-0,sj81PWrerDk,Primary Open-ended Question,Plot Attribute (Montage),What is the difference between the action of t...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
1,0008-1,sj81PWrerDk,Paraphrased Open-ended Question,Plot Attribute (Montage),Can you describe how the actions of the last p...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
2,0008-2,sj81PWrerDk,Correctly-led Open-ended Question,Plot Attribute (Montage),Did the last person open the bottle without us...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
3,0008-3,sj81PWrerDk,Wrongly-led Open-ended Question,Plot Attribute (Montage),Did the last person in the video open the bott...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
4,0008-7,sj81PWrerDk,Multiple-choice Question with a Single Correct...,Plot Attribute (Montage),How does the last person in the video open the...,8.85,E. None of the above\nSelect one best answer t...,,https://www.youtube.com/shorts/sj81PWrerDk


In [ ]:
whisper_model = whisper.load_model("large-v3")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

In [4]:
unique_videos = df[["video_id", "youtube_url"]].drop_duplicates()
print(f"Total unique videos: {len(unique_videos)}")

Total unique videos: 292


In [10]:
def download_video(video_id, youtube_url):
    video_path = f"videos/{video_id}.mp4"
    
    # Skip if already downloaded
    if os.path.exists(video_path):
        print(f"Skipping {video_id}, already downloaded.")
        return video_path

    try:
        command = f'yt-dlp -f bestvideo+bestaudio --merge-output-format mp4 "{youtube_url}" -o "{video_path}"'
        subprocess.run(command, shell=True, check=True)
        print(f"Downloaded: {video_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading {video_id}: {e}")
        with open("download_errors.log", "a") as f:
            f.write(f"{video_id},{youtube_url}\n")
        time.sleep(5)  # Wait before retrying the next video
    
    return video_path

# Process full dataset, handling errors
for i, row in unique_videos.iterrows():
    download_video(row["video_id"], row["youtube_url"])

[youtube] Extracting URL: https://www.youtube.com/shorts/sj81PWrerDk
[youtube] sj81PWrerDk: Downloading webpage
[youtube] sj81PWrerDk: Downloading tv client config
[youtube] sj81PWrerDk: Downloading player 6b3caec8
[youtube] sj81PWrerDk: Downloading tv player API JSON
[youtube] sj81PWrerDk: Downloading ios player API JSON
[youtube] sj81PWrerDk: Downloading m3u8 information
[info] sj81PWrerDk: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 2
[download] Destination: videos/sj81PWrerDk.f616.mp4


[download] 100% of    3.05MiB in 00:00:02 at 1.38MiB/s               
[download] Destination: videos/sj81PWrerDk.f251.webm
[download] 100% of  130.24KiB in 00:00:00 at 2.30MiB/s     
Downloaded: videos/sj81PWrerDk.mp4
[youtube] Extracting URL: https://www.youtube.com/shorts/AGCyLqLuUJ0
[youtube] AGCyLqLuUJ0: Downloading webpage
[youtube] AGCyLqLuUJ0: Downloading tv client config
[youtube] AGCyLqLuUJ0: Downloading player 6b3caec8
[youtube] AGCyLqLuUJ0: Downloading tv player API JSON
[youtube] AGCyLqLuUJ0: Downloading ios player API JSON
Error downloading AGCyLqLuUJ0: Command 'yt-dlp -f bestvideo+bestaudio --merge-output-format mp4 "https://www.youtube.com/shorts/AGCyLqLuUJ0" -o "videos/AGCyLqLuUJ0.mp4"' returned non-zero exit status 1.


ERROR: [youtube] AGCyLqLuUJ0: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


[youtube] Extracting URL: https://www.youtube.com/shorts/4mKxze5P9gQ
[youtube] 4mKxze5P9gQ: Downloading webpage
[youtube] 4mKxze5P9gQ: Downloading tv client config
[youtube] 4mKxze5P9gQ: Downloading player 6b3caec8
[youtube] 4mKxze5P9gQ: Downloading tv player API JSON
[youtube] 4mKxze5P9gQ: Downloading ios player API JSON
[youtube] 4mKxze5P9gQ: Downloading m3u8 information
[info] 4mKxze5P9gQ: Downloading 1 format(s): 247+251


[download] Destination: videos/4mKxze5P9gQ.f247.webm
[download] 100% of  193.94KiB in 00:00:00 at 1.34MiB/s     
[download] Destination: videos/4mKxze5P9gQ.f251.webm
[download] 100% of  241.05KiB in 00:00:01 at 152.32KiB/s   
Downloaded: videos/4mKxze5P9gQ.mp4
[youtube] Extracting URL: https://www.youtube.com/shorts/1SiTuYb506o
[youtube] 1SiTuYb506o: Downloading webpage



ERROR: Interrupted by user


KeyboardInterrupt: 

In [6]:
for i, row in unique_videos.iterrows():
    download_video(row["video_id"], row["youtube_url"])

[youtube] Extracting URL: https://www.youtube.com/shorts/sj81PWrerDk
[youtube] sj81PWrerDk: Downloading webpage
[youtube] sj81PWrerDk: Downloading tv client config
[youtube] sj81PWrerDk: Downloading player 6b3caec8
[youtube] sj81PWrerDk: Downloading tv player API JSON
[youtube] sj81PWrerDk: Downloading ios player API JSON
[youtube] sj81PWrerDk: Downloading m3u8 information
[info] sj81PWrerDk: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 2
[download] Destination: videos/sj81PWrerDk.f616.mp4


[download] 100% of    3.05MiB in 00:00:02 at 1.16MiB/s                 
[download] Destination: videos/sj81PWrerDk.f251.webm
[download] 100% of  130.24KiB in 00:00:00 at 1.03MiB/s   
[youtube] Extracting URL: https://www.youtube.com/shorts/AGCyLqLuUJ0
[youtube] AGCyLqLuUJ0: Downloading webpage
[youtube] AGCyLqLuUJ0: Downloading tv client config
[youtube] AGCyLqLuUJ0: Downloading player 6b3caec8
[youtube] AGCyLqLuUJ0: Downloading tv player API JSON
[youtube] AGCyLqLuUJ0: Downloading ios player API JSON


ERROR: [youtube] AGCyLqLuUJ0: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


CalledProcessError: Command 'yt-dlp -f bestvideo+bestaudio --merge-output-format mp4 https://www.youtube.com/shorts/AGCyLqLuUJ0 -o videos/AGCyLqLuUJ0.mp4' returned non-zero exit status 1.